In [42]:
import os
import networkx as nx
import itertools
import netlsd

def load_hypergraph(path, model):
    with open(path, 'r') as f:
        hg = f.readlines()
    if model == 'HyperDK00' or model == 'HyperDK11' or model == 'HyperPLR':
        hg = [list(map(int, e.split())) for e in hg]
    else:
        hg = [list(map(int, e.split(','))) for e in hg]
    return hg



def hg_bipartite(hg):
    G = nx.Graph()
    # Add nodes for the original vertices (left set)
    left_nodes = set(node for edge in hg for node in edge)
    G.add_nodes_from(left_nodes, bipartite=0)
    # Add nodes for the hyperedges (right set)
    right_nodes = [f'e{i}' for i in range(len(hg))]
    G.add_nodes_from(right_nodes, bipartite=1)
    # Add edges between vertices and their corresponding hyperedges
    for i, edge in enumerate(hg):
        for node in edge:
            G.add_edge(node, f'e{i}')

    return G

graph_path = './generate_graphs'

In [ ]:
desnity_result = {}

datasets = ['contact-high-school', 'contact-primary-school', 'email-Enron', 'email-Eu', 'NDC-classes']
models = ['Hyperlap', 'Hyperlap+', 'HyperPLR']

for dataset in datasets:
    for model in models:
        hgs_names = os.listdir(f'{graph_path}/{model}/{dataset}')
        hgs = [load_hypergraph(f'{graph_path}/{model}/{dataset}/{name}', model) for name in hgs_names]
        projections = [hg_bipartite(hg) for hg in hgs]

        total = 0
        for i in range(5):
            for j in range(i + 1, 5):
                desc1 = netlsd.heat(projections[i])
                desc2 = netlsd.heat(projections[j])
                distance = netlsd.compare(desc1, desc2)
                total += distance
        print(dataset, model, total)
        desnity_result[(dataset, model)] = total


In [ ]:
desnity_result_wave = {}
for dataset in datasets:
    for model in models:
        hgs_names = os.listdir(f'{graph_path}/{model}/{dataset}')
        hgs = [load_hypergraph(f'{graph_path}/{model}/{dataset}/{name}', model) for name in hgs_names]
        projections = [hg_bipartite(hg) for hg in hgs]
        total = 0
        for i in range(5):
            for j in range(i + 1, 5):
                desc1 = netlsd.wave(projections[i])
                desc2 = netlsd.wave(projections[j])
                distance = netlsd.compare(desc1, desc2)
                total += distance
        print(dataset, model, total)
        desnity_result_wave[(dataset, model)] = total


In [ ]:
print('(NetLSD-heat) | ' + ' | '.join(datasets))
print(' | '.join(['---'] * 6))
for model in models:
    print(model + ' | ' + ' | '.join([f'{desnity_result[(dataset, model)]:.3f}' for dataset in datasets]))

In [ ]:
print('(NetLSD-wave) | ' + ' | '.join(datasets))
print(' | '.join(['---'] * 6))
for model in models:
    print(model + ' | ' + ' | '.join([f'{desnity_result_wave[(dataset, model)]:.3f}' for dataset in datasets]))